In [2]:
import progressbar
import pickle

In [3]:
def write_dict(d):
    with open('data_concrete_rounding.pkl', 'wb') as file:
        pickle.dump(d, file)
        
def read_dict():
    with open('data_concrete_rounding.pkl', 'rb') as file:
        loaded_dict = pickle.load(file)
    return loaded_dict

# Concrete Formulas of Different Algorithms

##  (Rounding issues are considered)

#### Implementation of the cost estimates given in Section 4.6 and Appendix A.2

In [4]:
from math import log2, comb as binom, floor as fl, ceil as ce

def N(x):
    return x

def floor(x):
    """
    Floor of input $x$, i.e., largest integer smaller than $x$
    """
    return int(fl(x))

def ceil(x):
    """
    Ceil of input $x$, i.e., smallest integer greater than $x$
    """
    return int(ce(x))

def r_int(x):
    """
    Round of input $x$, i.e., integer closest to $x$
    """
    return int(round(x))


def binomial(a,b):
    """
    Computes $\\binom{a}{b}$
    """
    return binom(int(a),int(b))

def adapt_parameters(n,k,w):
    """
    Transform input instance into regular instance (in case $w$ does not divide $n$). 
    Outputs new RSD parameters
    """

    r = n - k; #initial redundancy
    b = floor(n/w); #length of unit vectors
    new_n = w*b; #code length of RSD instance
    new_k = new_n-r; #dimension of RSD instance

    return new_n, new_k, b;

def permutation_based_concrete_cost_rounding(n,k,w,b):
    """
    Concrete cost of permutation-based Regular ISD (Section 4.6)
    Rounding issues are considered
    """

    k_prime  = k-w; #add parity-checks

    #success pr
    w_f = w- (k_prime - w*fl(k_prime/w))
    w_c = k_prime - w*fl(k_prime/w)
    p_iter = (1-fl(k_prime/w)/b)**w_f * (1-ce(k_prime/w)/b)**w_c

    #cost of one iteration
    T_iter = n*(n-k_prime)**2;

    #overall cost
    cost = log2(T_iter)-log2(p_iter);

    return cost;


def enumeration_based_concrete_cost_rounding(n,k,w,b):
    """
    Concrete cost of Enumeration-based Regular ISD (Section A.2)
    Rounding issues are considered
    The function outputs a list with two elements: the first element gives the
    optimal setting for the algorithm (parameters $p$ and $\ell$, the second element
    is the corresponding time complexity
    """

    k_prime = k-w;  #add parity-checks

    min_cost = 1000000000000000000000000000000;
    best_params = [];


    p_max = 50;
    ell_max = 1000
    
    for p in range(0,min(p_max, fl(w/2))+1,2):
        for ell in range(0, min(ell_max, n-k_prime)+1):
            
            
            v = fl((k_prime+ell)/w)
                        
            if (v+1)>b:
                continue;
            
            w1 = w*(v+1)-k_prime-ell
            w2 = w-w1
            
            w1_minus = fl(w1/2)
            w2_minus = ce(w1/2)
            
            w1_plus = ce(w2/2)
            w2_plus = fl(w2/2)
            
            pr1 = 0; L1 = 0
            for i in range(max(0, fl(p/2)-w1_plus),min(fl(p/2),w1_minus)+1):
                term = binomial(w1_minus,i)*(1-v/b)**(w1_minus-i)*(v/b)**i*binomial(w1_plus,p/2-i)*(1-(v+1)/b)**(w1_plus-(p/2-i))*((v+1)/b)**(p/2-i);
                pr1+=term;
                L1 += binomial(w1_minus,i)*binomial(w1_plus,p/2-i)*v**i*(v+1)**(p/2-i)
                
                
            pr2 = 0; L2 = 0
            for i in range(max(0, fl(p/2)-w2_plus),min(fl(p/2),w2_minus)+1):
                term = binomial(w2_minus,i)*(1-v/b)**(w2_minus-i)*(v/b)**i*binomial(w2_plus,p/2-i)*(1-(v+1)/b)**(w2_plus-(p/2-i))*((v+1)/b)**(p/2-i);
                pr2+=term;
                L2 += binomial(w2_minus,i)*binomial(w2_plus,p/2-i)*v**i*(v+1)**(p/2-i)

            num_coll = L1*L2*2**(-ell);

            T_iter = n*(n-k_prime)**2+n*(L1+L2+num_coll);
            p_iter = pr1*pr2;
            
            if p_iter <= 0:
                continue;
                
            cost = log2(T_iter)-log2(p_iter);
            
            #update min cost  if new cost is smaller
            if cost<min_cost:
                min_cost = N(cost);
                best_params = {"p":p, "ell": ell};
                
                
    if best_params["p"]== p_max:
        print("!!!!ERROR: p_max is too low for enum-based");
    if best_params["ell"]== ell_max:
        print("!!!!ERROR: ell_max is too low for enum-based");
        
    return best_params, min_cost


###########################################################################################


# List of RSD parameters from the literature

In [5]:
#Format for all lists is [n, k, w, claimed_security_level]
#Parameters from https://eprint.iacr.org/2022/712.pdf
hardness_lpn_larger_weight = [[2**10, 652, 106, 146], [2**12, 1589, 172, 131], [2**14, 3482, 338, 132], [2**16, 7391, 667, 135], [2**18, 15336, 1312, 123], [2**20, 32771, 2467, 126], [2**22, 64770, 4788, 103]];
hardness_lpn_smaller_weight = [[2**10, 652, 57, 94], [2**12, 1589, 98, 83], [2**14, 3482, 198, 86], [2**16, 7391, 389, 91], [2**18, 15336, 760, 95], [2**20, 32771, 1419, 98], [2**22, 64770, 2735, 103]];

# Parameters  from Tables 1 and 2 of the paper https://eprint.iacr.org/2018/208.pdf 
tiny_keys_GMW = [[245760, 245460, 15],
 [1006632960, 1006631460, 15],
 [40960, 40660, 20],
 [85899345920, 85899342920, 20],
 [7680, 7380, 30],
 [1966080, 1964080, 30],
 [5120, 4720, 40],
 [1310720, 1307720, 40],
 [3200, 2750, 50],
 [12800, 11800, 50],
 [1280, 860, 80],
 [20480, 17980, 80]];
tiny_keys_GMW = [i+[128] for i in tiny_keys_GMW]

tiny_keys_BMR = [[42949672960, 42949671678, 10],
 [5905580032, 5905578870, 11],
 [4194304, 4193582, 16],
 [42949672960, 42949669758, 10],
 [2684354560, 2684351858, 20],
 [1638400, 1636798, 25],
 [10240, 9438, 40],
 [68719476736, 68719471614, 16],
 [25769803776, 25769798974, 24],
 [4194304, 4191582, 32],
 [14336, 13054, 56],
 [85899345920, 85899339518, 20],
 [1006632960, 1006627958, 30],
 [1310720, 1307718, 40],
 [15360, 13758, 60],
 [983040, 977438, 60],
 [25600, 22398, 100],
 [163840, 155038, 80],
 [30720, 24318, 120],
 [11520, 6718, 180]]
tiny_keys_BMR = [i+[128] for i in tiny_keys_BMR]

# Parameters from paper https://eprint.iacr.org/2019/1159.pdf
BCGIKRS = [[20000, 10000, 73, 80],
 [200000, 100000, 72, 80],
 [2000000, 1000000, 70, 80],
 [20000000, 10000000, 68, 80],
 [40000, 30000, 37, 80],
 [400000, 300000, 36, 80],
 [4000000, 3000000, 35, 80],
 [40000000, 30000000, 34, 80],
 [20000, 10000, 126, 128],
 [200000, 100000, 120, 128],
 [2000000, 1000000, 118, 128],
 [20000000, 10000000, 116, 128],
 [40000, 30000, 80, 128],
 [400000, 300000, 72, 128],
 [4000000, 3000000, 63, 128],
 [40000000, 30000000, 54, 128]]

# Parameters from the paper https://eprint.iacr.org/2022/1014.pdf (aggressive parameters)
BCGIKRS22 = [[2*2**i[0],2**i[0],i[1],128] for i in [[20,75],[25,72],[30,68]]]+[[5*2**i[0],2**i[0],i[1],128] for i in [[20,94],[25,90],[30,85]]]

# Parameters from https://eprint.iacr.org/2023/1035.pdf
ccj = [[1842, 825, 307, 128]];

# Parameters from https://eprint.iacr.org/2020/924.pdf
ferret=[[609728, 36288,1269,128],[10805248, 589760, 1319,128]]

# Set up Dictionary

In [6]:
all_params = {"PCG_large_weight": hardness_lpn_larger_weight,
 "PCG_small_weight": hardness_lpn_smaller_weight, 
 "tiny_keys_GMW":tiny_keys_GMW,
 "tiny_keys_BMR":tiny_keys_BMR,
 "BCGIKRS":BCGIKRS,
 "BCGIKRS22":BCGIKRS22,
 "ccj":ccj,
 "ferret":ferret}

In [7]:
all_estimates = {}

In [8]:
number_params=0
for i in all_params:
    c=1
    all_estimates[i]={}
    for j in all_params[i]:
        all_estimates[i][c]={}
        all_estimates[i][c]["params"]=j  
        all_estimates[i][c]["estimates"]={}
        all_estimates[i][c]["estimates"]["Perm_RoundingIssues"]={}
        all_estimates[i][c]["estimates"]["Enum_RoundingIssues"]={}
        c+=1
        number_params+=1
            

# Start costs estimate

### You can just load the dict if you want (instead of recomputing)

In [9]:
all_estimates=read_dict()

In [10]:
rows = [];
c=1

bar = progressbar.ProgressBar(maxval=number_params)
bar.start()
for i in all_estimates:
    for j in all_estimates[i]:
        
        
        n,k,w,claimed_security=all_estimates[i][j]["params"]

        #Adapt parameters
        new_n, new_k, b = adapt_parameters(n,k,w);

      
        if new_k/new_n<1:

            #Computing costs for regular ISD
            perm_cost = permutation_based_concrete_cost_rounding(new_n, new_k, w, b);
            enum_params, enum_cost = enumeration_based_concrete_cost_rounding(new_n, new_k, w, b);

            
            #Consider DOOM if code is quasi-cyclic
            if i == "BCGIKRS":
                p = n-k; #circulant size
                perm_cost = perm_cost - 0.5*log2(p);
                enum_cost = enum_cost - 0.5*log2(p);
            
           # print(n, k, w, "---->", r_int(perm_cost), r_int(enum_cost))
            all_estimates[i][j]["estimates"]["Perm_RoundingIssues"]["time"]=r_int(perm_cost)
            all_estimates[i][j]["estimates"]["Enum_RoundingIssues"]["time"]=r_int(enum_cost)
            all_estimates[i][j]["estimates"]["Enum_RoundingIssues"]["params"]=enum_params   

            bar.update(c)
            c+=1



# Printing

#### Numbers given in Appendix A.2

In [11]:
from prettytable import PrettyTable

max_rate=1
algorithms = [[i for i in all_estimates[j][1]["estimates"] if i!= "RepD2"] for j in all_estimates]
row_header = ["params"]+ algorithms

c=0
for i in all_estimates:
    print("\n\n")
    print("---------------------------------")
    print(i) 
    print("---------------------------------")
    row_header = ["params"]+ algorithms[c]
    t = PrettyTable(row_header)
    for j in all_estimates[i]:
        par = [all_estimates[i][j]["params"]]
        if par[0][1]/par[0][0]>max_rate:
            continue
        times = [all_estimates[i][j]["estimates"][k]["time"] for k in algorithms[c]]
        if j in ["PCG_large_weight", "PCG_small_weight"]:
            t.add_row(par+times)
        else:
            t.add_row(par+times)


    print(t)
    c+=1




---------------------------------
PCG_large_weight
---------------------------------
+-----------------------------+---------------------+---------------------+
|            params           | Perm_RoundingIssues | Enum_RoundingIssues |
+-----------------------------+---------------------+---------------------+
|    [1024, 652, 106, 146]    |         134         |         116         |
|    [4096, 1589, 172, 131]   |         132         |         111         |
|   [16384, 3482, 338, 132]   |         141         |         118         |
|   [65536, 7391, 667, 135]   |         149         |         126         |
|  [262144, 15336, 1312, 123] |         150         |         126         |
| [1048576, 32771, 2467, 126] |         164         |         139         |
| [4194304, 64770, 4788, 103] |         165         |         141         |
+-----------------------------+---------------------+---------------------+



---------------------------------
PCG_small_weight
----------------------

### Dump current dictionary to file

In [12]:
write_dict(all_estimates)